This version of the MTOC Workflow generates segmentations without label-free segmentions.

In [1]:
import numpy as np

# package for 3d visualization
from itkwidgets import view                              
from aicssegmentation.core.visual import seg_fluo_side_by_side,  single_fluorescent_view, segmentation_quick_view
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [16, 12]

# package for io 
from aicsimageio import AICSImage
from aicsimageio.writers import OmeTiffWriter

# function for core algorithm
from aicssegmentation.core.vessel import filament_3d_wrapper
from aicssegmentation.core.seg_dot import dot_2d
from aicssegmentation.core.seg_dot import dot_3d, dot_3d_wrapper 
from aicssegmentation.core.pre_processing_utils import intensity_normalization, edge_preserving_smoothing_3d, image_smoothing_gaussian_3d
from aicssegmentation.core.pre_processing_utils import intensity_normalization, image_smoothing_gaussian_slice_by_slice
from skimage.morphology import remove_small_objects 
from skimage.filters import threshold_otsu
from skimage.morphology import ball, binary_closing, remove_small_objects, dilation, erosion, disk, square, binary_dilation
from aicssegmentation.core.utils import topology_preserving_thinning, hole_filling
from aicssegmentation.core.MO_threshold import MO
from aicssegmentation.core.vessel import vesselness3D

import pandas as pd
from tifffile import imsave

In [7]:
manifest_csv = pd.read_csv('manifest.csv')

In [9]:
tubulin_only = manifest_csv[(manifest_csv['structure_name'] == 'TUBA1B')]

In [11]:
list(tubulin_only.raw_fov_path[10:30])

['/allen/aics/microscopy/Calysta/projects/training_emt/data/5500000406/ZSD0/alignV2/20210312_N02_002-alignV2-Scene-33-P33-D03.tiff',
 '/allen/aics/microscopy/Calysta/projects/training_emt/data/5500000406/ZSD0/alignV2/20210312_N02_002-alignV2-Scene-13-P13-D02.tiff',
 '/allen/aics/microscopy/Calysta/projects/training_emt/data/5500000406/ZSD0/alignV2/20210312_N02_002-alignV2-Scene-42-P42-D03.tiff',
 '/allen/aics/microscopy/Calysta/projects/training_emt/data/5500000406/ZSD0/alignV2/20210312_N02_002-alignV2-Scene-45-P45-D03.tiff',
 '/allen/aics/microscopy/Calysta/projects/training_emt/data/5500000406/ZSD0/alignV2/20210312_N02_002-alignV2-Scene-16-P16-D02.tiff',
 '/allen/aics/microscopy/Calysta/projects/training_emt/data/5500000406/ZSD0/alignV2/20210312_N02_002-alignV2-Scene-04-P4-D02.tiff',
 '/allen/aics/microscopy/Calysta/projects/training_emt/data/5500000406/ZSD0/alignV2/20210312_N02_002-alignV2-Scene-40-P40-D03.tiff',
 '/allen/aics/microscopy/Calysta/projects/training_emt/data/5500000406

<h1> MTOC TUBULIN WORKFLOW <h1>

In [2]:
#filename2 = '/allen/aics/microscopy/Calysta/projects/training_emt/data/5500000406/ZSD0/alignV2/20210312_N02_002-alignV2-Scene-32-P32-D03.tiff'
filename2 = "//allen/aics/assay-dev/users/Sandi/aics-segmentation/lookup_table_demo/tubulin/tubulin-50ms-imgs/raw/5500000704_EE_2-01_AcquisitionBlock7_pt7_Scene-0_aligned.ome.tiff"
reader = AICSImage(filename2) 
IMG2 = reader.data.astype(np.float32)
print(IMG2.shape)

Attempted file (//allen/aics/assay-dev/users/Sandi/aics-segmentation/lookup_table_demo/tubulin/tubulin-50ms-imgs/raw/5500000704_EE_2-01_AcquisitionBlock7_pt7_Scene-0_aligned.ome.tiff) load with reader: aicsimageio.readers.bfio_reader.OmeTiledTiffReader failed with error: No module named 'bfio'
The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).


(1, 4, 40, 1200, 1800)


In [3]:
#####################
## PARAMETER ##
structure_channel = 1
#####################

struct_img0 = IMG2[0,structure_channel,:,:,:].copy() 
#view(single_fluorescent_view(struct_img0))
struct_img0.shape

(40, 1200, 1800)

<h1> 1: PRE-PROCESSING <h1>

In [5]:
# intensity normalization v2
intensity_scaling_param2 = [6000] # original: 8000
struct_img2 = intensity_normalization(struct_img0, scaling_param=intensity_scaling_param2) #struct_img0


intensity norm: min-max norm with upper bound 6000


In [6]:
# gaussian smoothing
gaussian_smoothing_sigma2 = 1 # original: 1
structure_img_smooth2 = image_smoothing_gaussian_slice_by_slice(struct_img2, sigma=gaussian_smoothing_sigma2)

<h1> 2: CORE SEGMENTATION ALGORITHMS <h1>

In [7]:
# otsu thresholding: higher value, more segmentation
th = 4.5* threshold_otsu(structure_img_smooth2) # original at 1.4 # v4: 1.8 # v5: 1.6 prev: 3.4
overall_shape = dilation(
    remove_small_objects(structure_img_smooth2 > th, min_size=10, connectivity=1, in_place=False), #min_size = 10
    ball(1)
)
fixed_shape = erosion(
    hole_filling(overall_shape, hole_min=0, hole_max=5, fill_2d=True), #, hole_min= 0 hole_max = 5
    ball(1)
)

In [8]:
# 2d spot filter
s2 = np.zeros_like(fixed_shape)
for z in range(structure_img_smooth2.shape[0]):
    zslice = dot_2d(structure_img_smooth2[z,], 2)
    s2[z, :, :] = zslice > 0.08 #original: 0.03 v5 :0.02 
fixed_shape = np.logical_or(s2, fixed_shape)

In [9]:
#view(single_fluorescent_view(fixed_shape))

In [10]:
# hole filling step
seg = hole_filling(fixed_shape, hole_min=0, hole_max=15, fill_2d=True)# hole_min=0 # hole_max=15

In [11]:
# applying mask object threshold
bw2, object_for_debug = MO(seg, global_thresh_method='tri', object_minArea=1200, return_object=True)

Converting input from bool to <class 'numpy.uint8'> for compatibility.


In [12]:
thin_dist_preserve=0.01
thin_dist=2
bw_thin = topology_preserving_thinning(bw2>0, thin_dist_preserve, thin_dist)

<h1> 3: POST-PROCESSING <h1>

In [13]:
# Remove large area 
## PARAMETERS for this step ##
minArea = 30 # original 1
seg2 = remove_small_objects(bw_thin>0, min_size=minArea, connectivity=1, in_place=True)

In [14]:
# save the file
imsave("mtoc-200ms-sc0.tiff", seg2)
#d02s12
#d03s46
#d02s2

<h1> Extra Code (Not Currently in Use) <h1>

In [ ]:
# bw_binary = bw.astype(int)*1  

In [24]:
#img_norm = intensity_normalization(struct_img, scaling_param=[20, 25])# original: [2,20] #seg 12 & 14: [50,55] #seg 13: [30,35] seg14-2.0 [25 30] 14check3: [45 50] 14check4: 50 55
#img_smooth = edge_preserving_smoothing_3d(img_norm, numberOfIterations=30)

In [25]:
# #applying f3 filter and 3D wrapping

# # PARAMETERS for this step ##
# f3_param = [[1, 0.03]] # original: [[1, 0.01]]  v3: [[1.25, 0.02]]

# ###############################

# bw = filament_3d_wrapper(structure_img_smooth, f3_param) #structure_img_smooth
# bw = dot_3d_wrapper(structure_img_smooth, f3_param) #structure_img_smooth

In [ ]:
# # applying 3D Spot Filter
# s3_param = [[1, 0.009]] # v4: [[1, 0.05]] lower 2nd parameter, more seg
# ################################

# bw_extra = dot_3d_wrapper(structure_img_smooth, s3_param) # structure_img_smooth